In [2]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name = 'ts_backtest')

Found the config file in: C:\Users\honglu\TSPerf\prototypes\cross_validation\config.json
Performing interactive authentication. Please follow the instructions on the terminal.


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


In [11]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.core.conda_dependencies import CondaDependencies

env = EnvironmentDefinition()

# env.python.user_managed_dependencies = False
# env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'numpy', 'scikit-garden'], python_version='3.6.2')
# env.python.conda_dependencies.add_channel('conda-forge')
# env.docker.enabled=True

env.python.user_managed_dependencies = True
env.python.interpreter_path = 'C:/Users/honglu/AppData/Local/Continuum/Anaconda3/envs/tsperf/python.exe'

In [24]:
from azureml.train.estimator import Estimator

script_folder = './'

script_params = {
    '--data-folder': 'C:/Users/honglu/TSPerf/energy_load/GEFCom2017_D_Prob_MT_hourly/submissions/baseline/data/train_round_1.csv',
    '--n-estimators': 10,
    '--min-samples-split': 10
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target='local',
                entry_script='train_validation.py')

In [25]:
run = exp.submit(config=est)
from azureml.train.widgets import RunDetails
RunDetails(run).show()

_UserRun()

In [28]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

batchai_cluster_name = "hlutsperf"
try:
    compute_target = ComputeTarget(workspace=ws, name = batchai_cluster_name)
    if type(compute_target) is BatchAiCompute:
        print('found compute target {}, just use it.'.format(batchai_cluster_name))
    else:
        print('{} exists but it is not a Batch AI cluster. Please choose a different name.'.format(batchai_cluster_name))
except ComputeTargetException:
    print('creating a new compute target...')
    compute_config = BatchAiCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=0, 
                                                                cluster_max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, batchai_cluster_name, compute_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target.status.serialize()) 
    
BatchAiCompute.attach(workspace=ws,
                      name="hlutsperf",
                      resource_id="/subscriptions/ff18d7a8-962a-406c-858f-49acd23d6c01/resourceGroups/hluamlsdkrg/providers/Microsoft.BatchAI/workspaces/hlutsperf5f5171588c9/clusters/hlutsperf5f5171588c9")

# for ct in ws.compute_targets():
#     print(ct.name, ct.type, ct.provisioning_state)

In [55]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.core.conda_dependencies import CondaDependencies

env = EnvironmentDefinition()

env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'numpy', 'scikit-garden'], python_version='3.6.2')
env.python.conda_dependencies.add_channel('conda-forge')
env.docker.enabled=True

In [78]:
from azureml.core.runconfig import EnvironmentDefinition
script_folder = './'

script_params = {
    '--data-folder': ws.get_default_datastore().as_mount(),
    '--n-estimators': 10,
    '--min-samples-split': 10
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='train_validation.py',
                environment_definition=env)

In [79]:
run = exp.submit(config=est)

In [74]:
# ds = ws.get_default_datastore()
# ds.upload(src_dir='C:/Users/honglu/TSPerf/energy_load/GEFCom2017_D_Prob_MT_hourly/submissions/baseline/data/', target_path='tsbacktest', overwrite=True, show_progress=True)

In [81]:
from azureml.train.widgets import RunDetails
RunDetails(run).show()

_UserRun()

In [86]:
from azureml.train.hyperdrive import *
ps = RandomParameterSampling({
    '--min-samples-split': choice(5, 10),
    '--n-estimators': choice(10, 100)
})
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
htc = HyperDriveRunConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          primary_metric_name='average pinball loss', 
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                          max_total_runs=20,
                          max_concurrent_runs=4)
htr = exp.submit(config=htc)

C:\Users\honglu\AppData\Local\Continuum\Anaconda3\envs\tsperf\lib\site-packages\azureml\train\hyperdrive\runconfig.py:227: UserWarning: Same input parameter(s) specified in estimator script params and HyperDrive parameter space. HyperDrive parameter space definition will override duplicate entries in estimator. ['--n-estimators', '--min-samples-split'] is the list of overriden parameter(s).
  "{} is the list of overriden parameter(s).".format(duplicate_params))


_UserRun()

In [87]:
RunDetails(htr).show()

_HyperDrive(widget_settings={'childWidgetDisplay': 'popup'})